# Imports

Import custom files.

In [1]:
import area_comparison as ac
import general_functions as gf

Import other modules.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
import shapely

In [3]:
def IoU_poly_2(ref_poly, other_poly):    
    iou_polys = []
    for i in range(len(ref_poly)):       
        if not ref_poly[i] or not other_poly[i]:  # Prüft, ob Listen leer sind
            print(f"⚠️ Keine Polygone für Datei {i} gefunden, IoU = 0")
            iou_polys.append(0)
            continue

        try:
            ref_poly_1 = ref_poly[i][0]  # Erstes Polygon aus der Liste nehmen
            other_poly_1 = other_poly[i][0]  # Erstes Polygon aus der Liste nehmen
            
            # Berechne Schnitt- und Vereinigungsfläche
            intersection_area = ref_poly_1.intersection(other_poly_1).area
            union_area = ref_poly_1.union(other_poly_1).area

            # IoU berechnen, wobei Division durch 0 vermieden wird
            iou_poly = intersection_area / union_area if union_area > 0 else 0  
            iou_poly = round(iou_poly, 2)

        except Exception as e:
            print(f"❌ Fehler bei IoU-Berechnung für Datei {i}: {e}")
            iou_poly = 0

        iou_polys.append(iou_poly)

    return iou_polys

# Setup

Load the reference polygons from txt files:

In [4]:
path_to_references = '../outputs/evaluation/rcf/'
#only read files fitting to the filter. '*txt' would read all txt files in the folder
reference_contour_filter = '*mean_contour.txt'

In [5]:
reference_paths = glob.glob(path_to_references + reference_contour_filter)
reference_paths.sort()

Load the contours that should be compared with reference contours:

In [6]:
paths_to_compare_contours = ['../outputs/yolov8seg/rcf/', '../outputs/pyRES/', '../outputs/pyRES/']
#only read files fitting to the filter. '*txt' would read all txt files in the folder
compare_contour_filters = ['*YOLO.txt', '*HSV.txt', '*Sobel.txt']

In [7]:
compare_paths = []
for i in range(0, len(compare_contour_filters)):
    paths = glob.glob(paths_to_compare_contours[i] + compare_contour_filters[i])
    paths.sort()
    compare_paths.append(paths)

# Compare Areas by IoU

In [8]:
reference_classes, reference_masks= gf.read_masks_from_txt(reference_paths)
reference_polygons = [gf.polygons_from_coordinates([reference_masks[i][reference_classes[i].index(0)]])[0] for i in range(len(reference_masks))]
IoUs = []
for paths in compare_paths:
    compare_classes, compare_masks= gf.read_masks_from_txt(paths)
    compare_polygons = [gf.polygons_from_coordinates([compare_masks[i][compare_classes[i].index(0)]])[0] for i in range(len(compare_masks))]
    IoUs.append(IoU_poly_2(reference_polygons, compare_polygons))

In [9]:
IoUs

[[0.93, 0.97, 0.99, 0.96, 0.9],
 [0.03, 0.0, 0.22, 0.35, 0.07],
 [0.9, 0.0, 0.97, 0.93, 0.5]]

Mean and standard deviation per algorithm.

In [10]:
[(np.mean(algo), np.std(algo)) for algo in IoUs]

[(0.95, 0.03162277660168377),
 (0.13399999999999998, 0.13184839779079607),
 (0.66, 0.37089081951431474)]